#### Problem Statement

InsaidTelecom, one of the leading telecom players, understands that customizing offering
is very important for its business to stay competitive.
Currently, InsaidTelecom is seeking to leverage behavioral data from more than 60% of the 50 million mobile devices active daily in India to help its clients better understand and interact with their audiences.<br>

In this consulting assignment, Insaidians are expected to build a dashboard to understand
user's demographic characteristics based on their mobile usage, geolocation, and mobile device properties.
Doing so will help millions of developers and brand advertisers around the world pursue
data-driven marketing efforts which are relevant to their users and catered to their preferences.

#### Importing Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

#### Getting Evenets data from csv

In [ ]:
df_events_data=pd.read_csv('events_data.csv')

In [ ]:
df_events_data.shape

In [ ]:
df_events_data.isna().sum()

In [ ]:
df_events_data.info()

#### Getting gender_age_train & phone_brand_device_model data from mysql

In [ ]:
import mysql.connector

In [ ]:
dataBase = mysql.connector.connect(
  host ="cpanel.insaid.co",
  user ="student",
  passwd ="student",
  database = "Capstone1"
)

In [ ]:
# preparing a cursor object
cursorObject = dataBase.cursor()
genderagequery='SELECT * FROM gender_age_train'
cursorObject.execute(genderagequery)   
gender_age_train = cursorObject.fetchall()

In [ ]:
df_gender_age_train=pd.DataFrame(gender_age_train)

In [ ]:
df_gender_age_train.shape

In [ ]:
phonebrandquery='SELECT * FROM phone_brand_device_model'
cursorObject.execute(phonebrandquery)   
phone_brand_device_model = cursorObject.fetchall()

In [ ]:
df_phone_brand_device_model=pd.DataFrame(phone_brand_device_model)

In [ ]:
df_phone_brand_device_model.shape

In [ ]:
# disconnecting from server
dataBase.close()

#### EDA

In [ ]:
df_events_data.info(show_counts=True)

In [ ]:
df_events_data.isna().sum()

In [ ]:
df_events_data.head()

In [ ]:
df_events_data[df_events_data['state'].isna()]['city'].unique()

In [ ]:
df_events_data['state'].unique()

In [ ]:
def statenullvalueupdate(row):
    """This function is used to update stae column based on city column for null values"""
    if row['city']=='Pune':
        row['state']='Maharashtra'
    elif row['city']=='Visakhapatnam':
        row['state']='AndhraPradesh'
    elif row['city']=='Indore':
        row['state']='MadhyaPradesh'
    elif row['city']=='Chennai':
        row['state']='TamilNadu'
    elif row['city']=='Delhi':
        row['state']='Delhi'
    elif row['city']=='Channapatna':
        row['state']='Karnataka'
    elif row['city']=='Jaipur':
        row['state']='Rajasthan'
    elif row['city']=='Gangarampur':
        row['state']='WestBengal'
    elif row['city']=='Arambagh':
        row['state']='WestBengal'
    
    return row

In [ ]:
dfnull=df_events_data[df_events_data['state'].isna()]

In [ ]:
dfnull.shape

In [ ]:
dfnull.info()

In [ ]:
dfnull=dfnull.apply(lambda x:statenullvalueupdate(x),axis=1)

In [ ]:
dfnull.head()

In [ ]:
df_events_data.update(dfnull)

In [ ]:
df_events_data.info(show_counts=True)

In [ ]:
df_events_data.isna().sum()

In [ ]:
df_events_data.shape

In [ ]:
df_gender_age_train.info(show_counts=True)

In [ ]:
df_gender_age_train.isna().sum()

In [ ]:
df_gender_age_train.head()

In [ ]:
#Changing column names
df_gender_age_train.rename(columns={0:'device_id',1:'gender',2:'age',3:'age_group'},inplace=True)

In [ ]:
df_gender_age_train.head()

In [ ]:
df_gender_age_train['device_id'] =df_gender_age_train['device_id'].astype(float)

In [ ]:
df_gender_age_train.info()

In [ ]:
df_gender_age_train['age'].unique()

##### Box Plot for Age column to verify outliers

In [ ]:
plt.figure(figsize=(11,7))
sns.boxplot(x=df_gender_age_train['age'])
plt.show()

#### This code is to verify age_group and age correctly mapped. Based on the below query noticed that, age_group and age are correctly mapped

In [ ]:
agegrouplist_M=df_gender_age_train[df_gender_age_train['gender']=='M']['age_group'].value_counts().index.to_list()

In [ ]:
for i in agegrouplist_M:
    print(i)
    print(df_gender_age_train[df_gender_age_train['age_group']==i]['age'].value_counts())

In [ ]:
agegrouplist_F=df_gender_age_train[df_gender_age_train['gender']=='F']['age_group'].value_counts().index.to_list()

In [ ]:
for i in agegrouplist_F:
    print(i)
    print(df_gender_age_train[df_gender_age_train['age_group']==i]['age'].value_counts())

In [ ]:
df_phone_brand_device_model.info(show_counts=True)

In [ ]:
df_phone_brand_device_model.isna().sum()

In [ ]:
df_phone_brand_device_model.head()

In [ ]:
#Changing column names
df_phone_brand_device_model.rename(columns={0:'device_id',1:'brand',2:'model'},inplace=True)

In [ ]:
df_phone_brand_device_model.head()

In [ ]:
df_phone_brand_device_model['device_id'] =df_phone_brand_device_model['device_id'].astype(float)

In [ ]:
df_phone_brand_device_model.info()

##### Getting the data related to States MadhyaPradesh, Chhattisgarh, Uttaranchal, JammuandKashmir, Goa, Nagaland, as we need to concentrate on these states only

In [ ]:
eventsdata=df_events_data[df_events_data['state'].isin(['MadhyaPradesh', 'Chhattisgarh', 'Uttaranchal', 'JammuandKashmir', 'Goa', 'Nagaland'])]

In [ ]:
eventsdata.shape

In [ ]:
eventsdata.isna().sum()

In [ ]:
eventsdata.info()

#### Joining 3 tables using Inner join

In [ ]:
data=eventsdata.merge(df_gender_age_train,on='device_id',how='inner').merge(df_phone_brand_device_model,on='device_id',how='inner')

In [ ]:
data.shape

In [ ]:
data.isna().sum()

In [ ]:
data.info(show_counts=True)

#### Filling Null values for longitude and latitude

In [ ]:
data['longitude'].fillna(data[data['city']=='Indore']['longitude'].mode()[0],inplace=True)
data['latitude'].fillna(data[data['city']=='Indore']['latitude'].mode()[0],inplace=True)

In [ ]:
data.info(show_counts=True)

In [ ]:
data.head()

#### Below code is to replace all Chinese brand data to corresponding English 

In [ ]:
from translate import Translator
translator= Translator(from_lang="chinese",to_lang="english")

In [ ]:
import re
#data['brand_eng']=data['brand'].apply(lambda x:translator.translate(x))
#data['brand_eng']=data['brand'].apply(lambda x:translator.translate(x) if re.search("[\u4e00-\u9FFF]", x) else x)

In [ ]:
brand_dict={'vivo': 'vivo', 'HTC': 'HTC', 'OPPO': 'OPPO', '华为': 'Huawei', 'LG': 'LG', '优米': 'Quality rice',
            '三星': 'Samsung', '小米': 'Xiaomi', '魅族': 'Meizu', '联想 ': 'Lenovo', 'PPTV': 'PPTV', '酷派': 'Coolpad', 
            '乐视': 'LeEco', '锤子': 'Hammer', 'Lovme': 'Lovme', '努比亚': 'Nubia', 'TCL': 'TCL', '酷珀': 'Cooper', 
            '中国移动': 'China Mobile', '一加': 'one plus', '朵唯': 'Dove', '语信': 'language letter', '米歌': 'Mi Ge', 
            '酷比魔方': 'Cube Rubik&#39;s Cube', '富可视': 'InFocus', '青橙': 'Qing Cheng', '酷比': 'Cool than', 
            '首云': 'Shouyun', '夏新': 'Amoi', '乡米': 'rural rice', '波导': 'Waveguide', '天语': 'New SX4', '奇酷': 'cool', 
            '梦米': 'dream rice', '百加': 'Bacardi', '爱派尔': 'Apple', '摩托罗拉': 'Motorola', '诺基亚': 'Nokia', 
            '海信': 'Hisense', '易派': 'Epay', '华硕': 'ASUS', '青葱': 'green onion', '海尔': 'Haier', 
            '美图': 'beautiful picture', '维图': 'Dimensional', '黑米': 'black rice', '亿通': 'Billiontone', 
            '纽曼': 'Newmann', '先锋': 'Xianfeng Railway Station', '百立丰': 'Bailifeng', '飞秒': 'Femtosecond', 
            'E派': 'E Pie', '邦华': 'Banghua', '果米': 'fruit rice', '艾优尼': 'Ayuni', '欧博信': 'Obersin', '贝尔丰': 'Belfon', 
            '欧新': 'Ou Xin', 'SUGAR': 'SUGAR', '小杨树': 'small poplar'}

In [ ]:
data['brand_eng']=data['brand'].map(brand_dict)

#### Below code is to replace all Chinese model data to corresponding English 

#This is just a sample code to fill the dictionary 
model_dict_1 = {}

lst_model_1=['Desire 816x', 'L108', 'T87+', 'A670t',
'One E8', 'G610S', 'Y330', '麦芒3', 'R833T', 'R1C', 'X5Max', 'M032','X6', 'Desire 816', 'One X', 'Y18L', '小鲜2', '8089', '青春版']

for i in lst_model_1:
    if re.search("[\u4e00-\u9FFF]", i):
        translation = translator.translate(i)
    else:
        translation = i
    model_dict_1[i] = translation
    print(i, " ==> ", translation)

print(model_dict_1)

In [ ]:
model_dict={'Y928' : 'Y928','Sensation XE with Beats Audio Z715e' : 'Sensation XE with Beats Audio Z715e','R1' : 'R1',
'荣耀7' : 'glory 7','Y33' : 'Y33','G2' : 'G2','U705T' : 'U705T','R823T' : 'R823T','Galaxy Grand 2': 'Galaxy Grand 2',
'R831S' : 'R831S','MI 2' : 'MI 2','MX4' : 'MX4','荣耀畅玩4X' : 'Honor Play 4X','Galaxy S4' : 'Galaxy S4','X3S' : 'X3S','MI 4' : 'MI 4',
'红米note' : 'redmi note','乐檬K3 Note' : 'Lemeng K3 Note','Galaxy S6' : 'Galaxy S6','MI 2S' : 'MI 2S','A31' : 'A31','Xperia Z2' : 'Xperia Z2',
'R7' : 'R7','V889D' : 'V889D','荣耀3X畅玩版' : 'Honor 3X Play Edition','红米Note2' : 'redmi note2','King 7s' : 'King 7s','Y22L' : 'Y22L','X3T' : 'X3T',
'N1 Mini' : 'N1 Mini','Galaxy Note 2' : 'Galaxy Note 2','Galaxy Mega 5.8' : 'Galaxy Mega 5.8','Find 5' : 'Find 5','Ascend G7' : 'Ascend G7',
'Xperia SL' : 'Xperia SL','Galaxy J5' : 'Galaxy J5','GN9002' : 'GN9002','Galaxy S2' : 'Galaxy S2','Galaxy Note 3' : 'Galaxy Note 3','红米' : 'Red rice',
'荣耀X2' : 'Honor X2','超级手机1 Pro' : 'Super Phone 1 Pro','红米1S' : 'Redmi 1S','荣耀畅玩4' : 'Honor Play 4','Galaxy S5' : 'Galaxy S5',
'note顶配版' : 'note top version','MI 2A' : 'MI 2A','荣耀6' : 'glory 6','MI pad' : 'MI pad','MI 3':'MI 3','6607':'6607','Galaxy S4 Zoom' : 'Galaxy S4 Zoom',
'MX2' : 'MX2Z','Galaxy S3' : 'Galaxy S3','MX3' : 'MX3','Ascend P7' : 'Ascend P7','S6' : 'S6','MX5' : 'MX5','MI 2C' : 'MI 2C',
'Desire 7088' : 'Desire 7088','UIMI4' : 'UIMI4','Y11' : 'Y11','小米note' : 'Xiaomi note','Y27' : 'Y27','ELIFE E3T' : 'ELIFE E3T','Xplay' : 'Xplay',
'魅蓝metal' : 'Charm blue metal','魅蓝2' : 'Charm Blue 2','8729' : '8729','Galaxy A7' : 'Galaxy A7','魅蓝NOTE' : 'Charm Blue NOTE',
'红米note增强版' : 'Redmi Note Enhanced Edition','荣耀3C' : 'Honor 3C','超级手机1s' : 'Super phone 1s','Galaxy Grand' : 'Galaxy Grand','T1' : 'T1',
'I8258' : 'I8258','MI One C1' : 'MI One C1','Galaxy Style DUOS' : 'Galaxy Style DUOS','Y22IL' : 'Y22IL','A800' : 'A800','G520-0000' : 'G520-0000',
'S7' : 'S7','红米2' : 'red rice 2','Galaxy S6 Edge+' : 'Galaxy S6 Edge+','B199' : 'B199','R7 Plus' : 'R7 Plus','X3L' : 'X3L','Galaxy Win' : 'Galaxy Win',
'Mate 7' : 'Mate 7','A788t' : 'A788t','F2' : 'F2','麦芒4' : 'Maimang 4','T9' : 'T9','A820' : 'A820','Xperia arc S' : 'Xperia arc S','Q505T' : 'Q505T',
'GN139' : 'GN139','3007' : '3007','R2017' : 'R2017','7620l' : '7620l','大神F1' : 'Great God F1','X1ST' : 'X1ST','T328w' : 'T328w','5891' : '5891',
'Y17T' : 'Y17T','A330e' : 'A330e','Y13L' : 'Y13L','8190' : '8190','Z7 Mini' : 'Z7 Mini','R827T' : 'R827T','Ascend GX1' : 'Ascend GX1','N1T' : 'N1T',
'Xperia Z' : 'Xperia Z','联想黄金斗士S8' : 'Lenovo Gold Fighter S8','Nexus 5' : 'Nexus 5','Mate 8' : 'Mate 8','Y511-T00' : 'Y511-T00',
'魅蓝Note 2' : 'Meizu Note 2','荣耀6 Plus' : 'Honor 6 Plus','MI 5' : 'MI 5','G620-L75' : 'G620-L75','5892' : '5892','么么哒' : 'Meme da',
'Galaxy S Advance' : 'Galaxy S Advance','X5Pro' : 'X5Pro','Ascend G700' : 'Ascend G700','Xperia S' : 'Xperia S','Xplay3S' : 'Xplay3S','R3' : 'R3',
'Z5S Mini' : 'Z5S Mini','Galaxy S6 Edge' : 'Galaxy S6 Edge','R7s' : 'R7s','SM-T2556' : 'SM-T2556','Y913' : 'Y913','V185' : 'V185','锋尚' : 'Frontier',
'A53' : 'A53','Galaxy Trend 3' : 'Galaxy Trend 3','Ascend P6' : 'Ascend P6','7105' : '7105','红米2A' : 'redmi 2A','荣耀畅玩5X' : 'Honor Play 5X',
'S5.1' : 'S5.1','R821T' : 'R821T','Xperia SP' : 'Xperia SP','7296' : '7296','7295+' : '7295+','7269' : '7269','X5L' : 'X5L','7295C' : '7295C',
'Galaxy Note' : 'Galaxy Note','8190Q' : '8190Q','G3818' : 'G3818','X5Max+' : 'X5Max+','MI 1S' : 'MI 1S','荣耀畅玩4C' : 'Honor Play 4C',
'Galaxy Mega Plus' : 'Galaxy Mega Plus','Galaxy Premier' : 'Galaxy Premier','L7' : 'L7','P8 Lite' : 'P8 Lite','C8815' : 'C8815','A1' : 'A1','Y500' : 'Y500',
'A11' : 'A11','Two' : 'Two','R5' : 'R5','MI One Plus' : 'MI One Plus','Galaxy S2 Plus' : 'Galaxy S2 Plus','Y325' : 'Y325','A708t' : 'A708t','Y35' : 'Y35',
'X5SL' : 'X5SL','G610T' : 'G610T','Ascend P8' : 'Ascend P8','GN705' : 'GN705','E8' : 'E8','Galaxy S3 Neo+' : 'Galaxy S3 Neo+','G717C' : 'G717C',
'G750-T20' : 'G750-T20','Vibe Shot' : 'Vibe Shot','G730-U00' : 'G730-U00','L1' : 'L1','C8816' : 'C8816','5263' : '5263','7231' : '7231',
'Galaxy Trend DUOS 2' : 'Galaxy Trend DUOS 2','X5M' : 'X5M','Y23L' : 'Y23L','Galaxy Note 3 Lite' : 'Galaxy Note 3 Lite','7260' : '7260','One' : 'One',
'Galaxy Mega 6.3' : 'Galaxy Mega 6.3','Desire 610t' : 'Desire 610t','T329t' : 'T329t','N3' : 'N3','F103' : 'F103','Xperia Z1':'Xperia Z1','Y13iL' : 'Y13iL',
'Galaxy Grand DUOS' : 'Galaxy Grand DUOS','C8818' : 'C8818','M8' : 'M8','荣耀4A' : 'Honor 4A','Blade S6' : 'Blade S6','E6T' : 'E6T','A51' : 'A51',
'小辣椒 M2' : 'little tomboy','大器2' : 'Big 2','乐玩' : 'have fun','R1S' : 'R1S','小米4C' : 'Xiaomi 4C','3' : '3','795+' : '795+','M201S' : 'M201S',
'U707T' : 'U707T','A3000-H' : 'A3000-H','G7200' : 'G7200','7251' : '7251','MX4 Pro' : 'MX4 Pro','Find 7' : 'Find 7','Z5S' : 'Z5S','Y51' : 'Y51',
'9976A' : '9976A','G615-U10' : 'G615-U10','T35' : 'T35','Galaxy E7' : 'Galaxy E7','A860e' : 'A860e','8750' : '8750','Y635' : 'Y635','1107' : '1107',
'A850' : 'A850','M8St' : 'M8St','R8205' : 'R8205','荣耀畅玩4C运动版' : 'Honor Play 4C Sports Edition','8085' : '8085','Y15' : 'Y15','M512' : 'M512',
'N2' : 'N2','E7' : 'E7','R6007' : 'R6007','Q802T' : 'Q802T','Y927' : 'Y927','Galaxy Note 4' : 'Galaxy Note 4','M310' : 'M310','Galaxy On5' : 'Galaxy On5',
'C8813Q' : 'C8813Q','M812C' : 'M812C','R819T' : 'R819T','U701' : 'U701','A33' : 'A33','Y29L' : 'Y29L','5872' : '5872',
'大神F2全高清版':'Great God F2 full HDversion','Nexus S' : 'Nexus S','Galaxy A5' : 'Galaxy A5','G660-L075' : 'G660-L075','GN151' : 'GN151',
'Y600-U00' : 'Y600-U00','G3609' : 'G3609','X1' : 'X1','One M8' : 'One M8','M100' : 'M100','A850+' : 'A850+','S850' : 'S850','红米Note3' : 'redmi note3',
'G730-T00' : 'G730-T00','Galaxy J7' : 'Galaxy J7','V5 Max' : 'V5 Max','D350' : 'D350','荣耀畅玩5' : 'Honor Play 5','R831T' : 'R831T',
'Galaxy S Plus' : 'Galaxy S Plus','5956' : '5956','P301M' : 'P301M','A820t' : 'A820t','荣耀7i' : 'Honor 7i','A3800d' : 'A3800d','Y511-U00' : 'Y511-U00',
'Y13' : 'Y13','Ascend G700T':'Ascend G700T','S898t':'S898t','A830' : 'A830','A320t' : 'A320t','A678t' : 'A678t','Galaxy Grand 2 LTE':'Galaxy Grand 2 LTE',
'Y628' : 'Y628','Xperia C3' : 'Xperia C3','SHV-E210L' : 'SHV-E210L','Nexus 4' : 'Nexus 4','X710L' : 'X710L','X907' : 'X907','K860i' : 'K860i',
'Galaxy A3' : 'Galaxy A3','S7T' : 'S7T','One ME' : 'One ME','V955' : 'V955','C8817E' : 'C8817E','T5' : 'T5','P1' : 'P1','VIBE Z' : 'VIBE Z',
'Galaxy Core Prime' : 'Galaxy Core Prime','R809T' : 'R809T','V5S' : 'V5S','X6 Plus D' : 'X6 Plus D','TALK 9X' : 'TALK 9X','8730L' : '8730L','T20L':'T20L',
'S868t' : 'S868t','N821' : 'N821','X8888' : 'X8888','L5Pro' : 'L5Pro','X6 Plus' : 'X6 Plus','乐檬K3' : 'Lemont K3','Galaxy Ace 3' : 'Galaxy Ace 3',
'A388t' : 'A388t','Mate 2' : 'Mate 2','Desire 816x': 'Desire 816x', 'L108': 'L108', 'T87+': 'T87+', 'A670t': 'A670t', 'One E8': 'One E8', 'G610S': 
'G610S', 'Y330': 'Y330', '麦芒3': 'Maimang 3', 'R833T': 'R833T', 'R1C': 'R1C', 'X5Max': 'X5Max', 'M032': 'M032', 'X6': 'X6', 
'Desire 816': 'Desire 816', 'One X': 'One X', 'Y18L': 'Y18L', '小鲜2': 'Small fresh 2', '8089': '8089', '青春版': 'Youth Edition',
 'Z9 mini' : 'Z9 mini', 'A805e' : 'A805e', 'A889' : 'A889', 'A658t' : 'A658t', 'S5.5L' : 'S5.5L', 'M5' : 'M5',
'5950' : '5950', 'S5000' : 'S5000', 'S9' : 'S9', 'P6' : 'P6', 'A890e' : 'A890e', 'U86' : 'U86', 'Y19T' : 'Y19T',
'X6 D' : 'X6 D', 'S7I' : 'S7I', '5216D' : '5216D', 'R815T' : 'R815T', 'F301' : 'F301', 'V6' : 'V6',
'超级手机1' : 'super phone 1', 'U9508' : 'U9508', '大神F1Plus' : 'Great God F1Plus', 'Y13T' : 'Y13T', 'S6T' : 'S6T',
'Y300' : 'Y300', 'HN3-U01' : 'HN3-U01', 'Galaxy Grand Prime' : 'Galaxy Grand Prime', 'T9608' : 'T9608',
'A690' : 'A690', '红辣椒' : 'Chili pepper', '黄金斗士A8' : 'Gold Fighter A8', 'iPh-800' : 'iPh-800',
'坚果手机' : 'Nut phone', 'My 布拉格' : 'My Prague', 'MT1-U06' : 'MT1-U06', 'Galaxy Trend DUOS' : 'Galaxy Trend DUOS',
'G3588V' : 'G3588V', 'F105' : 'F105', 'Galaxy Nexus' : 'Galaxy Nexus', '三星big foot' : 'foot', 'A828t' : 'A828t',
'A880' : 'A880', 'XT910' : 'XT910', 'P8max' : 'P8max', 'GN715' : 'GN715', '8702' : '8702', 'Y11IW' : 'Y11IW',
'802w' : '802w', 'N1' : 'N1', 'Galaxy Core Max' : 'Galaxy Core Max', 'H3' : 'H3', 'C8650' : 'C8650', 'A399' : 'A399',
'X1S' : 'X1S', 'Z7 Max' : 'Z7 Max', 'Y17W' : 'Y17W', 'Y11IT' : 'Y11IT', 'Galaxy Grand Neo Plus' : 'Galaxy Grand Neo Plus',
'7298D' : '7298D', 'XL' : 'XL', 'G11' : 'G11', 'HS-U978' : 'HS-U978', 'D500' : 'D500', '魅蓝' : 'Meilan',
'S11T' : 'S11T', 'E5' : 'E5', 'Galaxy Win Pro' : 'Galaxy Win Pro', 'Z9 Max' : 'Z9 Max', 'A3' : 'A3', 'S2L' : 'S2L',
'X1 7.0' : 'X1 7.0', 'A628t' : 'A628t', 'Desire 820' : 'Desire 820', 'Galaxy Trend 2' : 'Galaxy Trend 2',
'U807' : 'U807', 'PRO5' : 'PRO5', 'ZenFone Max' : 'ZenFone Max', 'S7568' : 'S7568', 'VIBE Z2' : 'VIBE Z2',
'Xperia Z3' : 'Xperia Z3', 'X5V' : 'X5V', 'Moto G' : 'Moto G', 'R830' : 'R830', 'A680' : 'A680', 'P520L' : 'P520L',
'I8268' : 'I8268', 'G716' : 'G716', 'A798t' : 'A798t', 'metal 标准版' : 'Line metal', 'V183' : 'V183',
'7270' : '7270', 'G610C' : 'G610C', 'Galaxy Core Mini' : 'Galaxy Core Mini', 'S880i' : 'S880i', 'G606' : 'G606',
'A656' : 'A656', 'U950' : 'U950', 'A560' : 'A560', 'HT-I860' : 'HT-I860', 'Galaxy Core 2' : 'Galaxy Core 2',
'A2580' : 'A2580', '606w' : '606w', 'A688t' : 'A688t', 'S5830I' : 'S5830I', 'C8816D' : 'C8816D', 'X805' : 'X805',
'ELIFE E6 mini' : 'ELIFE E6 mini', 'C986t' : 'C986t', 'Desire 826' : 'Desire 826', 'Xperia C' : 'Xperia C',
'G510' : 'G510', 'Galaxy Core Lite' : 'Galaxy Core Lite', '荣耀U8860' : 'Honor U8860', 'M4' : 'M4', 'UIMI3' : 'UIMI3',
'A360t' : 'A360t', 'VT898' : 'VT898', 'Xperia Z Ultra' : 'Xperia Z Ultra', 'G525' : 'G525', 'Galaxy On7' : 'Galaxy On7',
'ELIFE S5.5' : 'ELIFE S5.5', 'Galaxy Ace Dear' : 'Galaxy Ace Dear', '大神F2' : 'Great God F2', 'Y80D' : 'Y80D',
'Galaxy Tab 3 8.0' : 'Galaxy Tab 3 8.0', 'MI 4S' : 'MI 4S', 'ivvi 小i' : 'ivvi small i', 'Galaxy Core 4G' : 'Galaxy Core 4G',
'A199' : 'A199', '5316' : '5316', 'Galaxy Note 8.0' : 'Galaxy Note 8.0', 'K900' : 'K900', 'M1' : 'M1', 'Galaxy A8' : 'Galaxy A8',
'N919' : 'N919', 'U960S3' : 'U960S3', 'A760' : 'A760', 'R813T' : 'R813T', 'G7 Plus' : 'G7 Plus', 'L1 mini' : 'L1 mini',
'P3' : 'P3', 'U879' : 'U879', 'A8-50' : 'A8-50', '旗舰版' : 'Ultimate', 'Butterfly' : 'Butterfly', 'Y28L' : 'Y28L',
'A278t' : 'A278t', 'Galaxy S3 Mini' : 'Galaxy S3 Mini', '8720L' : '8720L', 'S2y' : 'S2y', 'Galaxy Mega 2' : 'Galaxy Mega 2',
'纽扣' : 'buttons', '7270-W00' : '7270-W00', 'A766' : 'A766', 'U817' : 'U817', 'Touch 2' : 'Touch 2', '9976D' : '9976D',
'K88L' : 'K88L', '荣耀6 plus' : 'honor 6 plus', 'T8830Pro' : 'T8830Pro', 'idol3' : 'idol3', 'A355e' : 'A355e', '2' : '2',
'Y610-U00' : 'Y610-U00', 'Moto X' : 'Moto X', 'A3900' : 'A3900', 'Galaxy Note Edge' : 'Galaxy Note Edge', 'MT2-C00' : 'MT2-C00',
'S720i' : 'S720i', 'Y635-TL00' : 'Y635-TL00', '8712' : '8712', 'F303' : 'F303', '5200S' : '5200S', 'S720' : 'S720',
'Galaxy Tab P1000' : 'Galaxy Tab P1000', 'A300t' : 'A300t', 'GN152' : 'GN152', 'Y321C' : 'Y321C', 'V967S' : 'V967S',
'5890' : '5890', 'A378t' : 'A378t', 'Galaxy S2 HD LTE E120S' : 'Galaxy S2 HD LTE E120S', 'Y20T' : 'Y20T', 'I9050' : 'I9050',
'8295M' : '8295M', '金钢' : 'Gold Steel', 'A789' : 'A789', 'H1' : 'H1', 'Grand S 2' : 'Grand S 2', 'A630t' : 'A630t',
'S12' : 'S12', '5891Q' : '5891Q', 'N1 max' : 'N1 max', 'S3' : 'S3', 'E3' : 'E3', 'Mate S' : 'Mate S', 'L8' : 'L8',
'Xperia TX' : 'Xperia TX', 'M601' : 'M601', '8017-T00' : '8017-T00', 'G520-5000' : 'G520-5000', 'X10i' : 'X10i',
'Butterfly s' : 'Butterfly s', 'Y320T' : 'Y320T', '7295' : '7295', 'Y516' : 'Y516', 'P70' : 'P70', 'A269i' : 'A269i',
'5951' : '5951', 'Y320' : 'Y320', 'M20-T' : 'M20-T', '9190l' : '9190l', 'N1W' : 'N1W', 'W2014' : 'W2014', 'T2' : 'T2',
'W999' : 'W999', 'Touch 3' : 'Touch 3', 'M811' : 'M811', 'T7' : 'T7', 'G3819D' : 'G3819D', 'EG970' : 'EG970',
'R817' : 'R817', '8713' : '8713', 'G730-C00' : 'G730-C00', 'iSuper S2' : 'iSuper S2', 'Xperia T2 Ultra' : 'Xperia T2 Ultra',
'U930' : 'U930', 'E6' : 'E6', 'V70' : 'V70', 'P331M' : 'P331M', 'P880' : 'P880', 'D800' : 'D800', 'I9118' : 'I9118',
'X68T' : 'X68T', 'Q701C' : 'Q701C', 'U701T' : 'U701T', 'B9388' : 'B9388', 'ZenFone 5' : 'ZenFone 5', 'Xperia U' : 'Xperia U',
'Galaxy W' : 'Galaxy W', 'Mate' : 'Mate', 'M6' : 'M6', 'Galaxy Tab S T805C' : 'Galaxy Tab S T805C', 'GN708T' : 'GN708T',
'VT888' : 'VT888', 'S7898' : 'S7898', 'Galaxy Tab 3 7.0' : 'Galaxy Tab 3 7.0', 'X3V' : 'X3V', '9180' : '9180',
'MediaPad M2-803L' : 'MediaPad M2-803L', 'Galaxy Ace' : 'Galaxy Ace', 'Y37' : 'Y37', 'U3' : 'U3', 'Xshot' : 'Xshot',
'Y923' : 'Y923', 'R811' : 'R811', 'A390t' : 'A390t', 'C8813' : 'C8813', 'A375e' : 'A375e', 'U956' : 'U956', 'A785e' : 'A785e',
'G4' : 'G4', 'Z5 Mini' : 'Z5 Mini', 'L22' : 'L22', 'Q302C' : 'Q302C', 'Optimus G Pro' : 'Optimus G Pro', 'S850t' : 'S850t',
'U818' : 'U818', '8722' : '8722', 'U51GT-W' : 'U51GT-W', 'GN180' : 'GN180', 'U59GT' : 'U59GT', 'S890' : 'S890', 'V4' : 'V4',
'S899t' : 'S899t', 'A385e' : 'A385e', 'GN708W' : 'GN708W', 'A238t' : 'A238t', 'GN700W' : 'GN700W', 'IVO 6655' : 'IVO 6655',
'Galaxy Alpha' : 'Galaxy Alpha', 'A858t' : 'A858t', 'BF A500' : 'BF A500', 'S7566' : 'S7566', 'A750e' : 'A750e', '春雷HD' : 'Spring Thunder HD',
'A6800' : 'A6800', 'One S' : 'One S', 'GN5001S' : 'GN5001S', 'T80' : 'T80', 'A318t' : 'A318t', 'T708' : 'T708', 'S820' : 'S820', 'U930HD' : 'U930HD',
'Desire 626' : 'Desire 626', '7268' : '7268', 'M3' : 'M3', 'G6-C00' : 'G6-C00', 'Galaxy Tab 3 10.1' : 'Galaxy Tab 3 10.1',
'Xperia Z5 Premium' : 'Xperia Z5 Premium', '8705' : '8705', 'Y220T' : 'Y220T', 'IdeaTab A1010-T' : 'IdeaTab A1010-T',
'Optimus LTE' : 'Optimus LTE', '8122' : '8122', 'MM1101' : 'MM1101', 'Xperia T' : 'Xperia T',
'A2800d' : 'A2800d', '小辣椒 4' : 'little tomboy', '5219' : '5219', 'Mate 7 青春版' : 'Mate 7 Youth Edition', 
'Galaxy Ace Plus' : 'Galaxy Ace Plus', 'Ascend G6' : 'Ascend G6','7295A青春版' : '7295A Youth Edition','小鲜3' : 'Small Umami3','大神Note3' : 'Great Gods Note3',
'红米3' : 'Red rice 3','荣耀3C畅玩版' : 'Rongyao 3C Enjoy 150 unique and brain-teasing levels. Version',
'锋尚Pro' : 'Frontier Pro','黄金斗士Note8' : 'HUANG Jin Battle Knight Note8','天鉴W808' : 'heavenly mirror W808',
'2016版 Galaxy A9' : '2016 Version Galaxy A9','倾城L3' : 'Fallen City L3','醉享' : 'drunk…Enjoy','大观4' : 'Great Guan 4',
'星星2号' : 'Star2.','红辣椒Note' : 'Chili pepper Note','锋尚2' : 'Frontier 2','畅享5S' : 'Smooth communication, cohesion and friendship Enjoy 5S',
'时尚手机' : 'Stylish Smartphone','麦芒3S' : 'wheat tip 3S','么么哒3N' : 'Meme da 3N','2016版 Galaxy A7' : '2016 Version Galaxy A7',
'红辣椒任性版 Plus' : 'Chili pepper capricious Version Plus','超级手机Max' : 'Exceed Grade Smartphone Max','大神Note' : '大Great God Note',
'荣耀3X' : 'Rongyao 3X','炫影S+' : 'Aion S 2019 Xuan 630 Shadow S+','红牛V5' : 'RedCowV5','TALK 7X四核' : 'TALK 7X Quad core','小辣椒S1' : 'little tomboy S1'}

In [ ]:
data['model_eng']=data['model'].map(model_dict)

#### Saving a Copy of Original Dataframe, just for saferside.

In [ ]:
data_copy=data.copy(deep=True)

#### Plotting Folium map to verify discrepancies in latitude and longitude (as per communcation from INSAID)

In [ ]:
import folium

In [ ]:
dfsample=data.groupby(['state','city','latitude','longitude']).count().reset_index()

In [ ]:
map_osm = folium.Map(location=[21.7679, 78.8718], zoom_start=2.5, tiles="Stamen Toner")
dfsample.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]], 
                                              radius=10,tooltip=row['state']+'\n'+row['city'],color="red",
                                              popup=str(row['latitude']) + '\n'+str(row['longitude']))
                                             .add_to(map_osm), axis=1)
map_osm

### Observation
As per the above map we noticed that, three latitude and longitude having discrepancies.<br>
Below are the three values related to latitude and longitude values which are having discrepancy.<br>
41.8719 12.5674<br>
25.2048 55.2708<br>
34.5553 69.2075

In [ ]:
data[(data['latitude'].isin([41.8719,25.2048,34.5553])) & (data['longitude'].isin([12.5674,55.2708,69.2075]))]

#### Replacing latitude and longitude values using Mode of Indore city

In [ ]:
lst=data[(data['latitude'].isin([41.8719,25.2048,34.5553])) & (data['longitude'].isin([12.5674,55.2708,69.2075]))].index

In [ ]:
data[data['city']=='Indore']['latitude'].mode()[0]

In [ ]:
for i in lst:
    data.at[i,'latitude']=data[data['city']=='Indore']['latitude'].mode()[0]
    data.at[i,'longitude']=data[data['city']=='Indore']['longitude'].mode()[0]

In [ ]:
data[(data['latitude'].isin([41.8719,25.2048,34.5553])) & (data['longitude'].isin([12.5674,55.2708,69.2075]))]

In [ ]:
dfsample=data.groupby(['state','city','latitude','longitude']).count().reset_index()

#### Below is the map after values updated

In [ ]:
map_osm = folium.Map(location=[21.7679, 78.8718], zoom_start=2.5, tiles="Stamen Toner")
dfsample.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]], 
                                              radius=10,tooltip=row['state']+'\n'+row['city'],color="red",
                                              popup=str(row['latitude']) + '\n'+str(row['longitude']))
                                             .add_to(map_osm), axis=1)
map_osm

In [ ]:
map_osm = folium.Map(location=[21.7679, 78.8718], zoom_start=4.5, tiles="Stamen Toner")
dfsample.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]], 
                                              radius=10,tooltip=row['state']+'\n'+row['city'],color="red",
                                              popup=str(row['latitude']) + '\n'+str(row['longitude']))
                                             .add_to(map_osm), axis=1)
map_osm

### Below code is to split the timestamp column to day, weekday, month,month-name and hour columns

In [ ]:
import datetime
data['timestamp'] = pd.to_datetime(data['timestamp'], format='%Y-%m-%d')
#data['Year']=data['timestamp'].dt.year   #---- We have data for only one year 2016. So, year column may not required.
data['Month']=data['timestamp'].dt.month
data['MonthName']=data['timestamp'].dt.strftime('%B')
data['Day']=data['timestamp'].dt.day
data['Weekday']=data['timestamp'].dt.strftime('%A')
data['Hour']=data['timestamp'].dt.strftime('%H')

In [ ]:
data.sample(3)

#### Converting device_id data type to int as the current format is not readable/exponencial format.

In [ ]:
data= data.astype({'device_id':'int64'})

#### Below code is to confrim a device_id fallin more than one state 

In [ ]:
df_state=data.groupby(by='device_id').agg({'state':'nunique'}).reset_index()

In [ ]:
df_state[df_state['state']==1]

Conclision:- it is confirmed that, any device_id fallin only one state.

### 1. Distribution of Users(device_id) across States.

In [ ]:
df_decicestate=data.groupby(by=['device_id','state']).agg({'event_id':'count'}).reset_index()

In [ ]:
plt.figure(figsize=(18,11))
sns.histplot(x=df_decicestate['state'],data=df_decicestate)
plt.title('Distribution of Users(device_id) across States.',fontsize=20)
plt.xlabel('State',fontsize=18,color='red')
plt.ylabel('No of records',fontsize=14)
plt.xticks(fontsize=14)
plt.show()

### 2. Distribution of Users across Phone Brands(Consider only 10 Most used Phone Brands).

In [ ]:
df_brand=data.groupby(by=['brand_eng']).agg({'device_id':'nunique'}).reset_index().sort_values(by='device_id',ascending=False)[:10]

In [ ]:
plt.figure(figsize=(18,11))
sns.barplot(x=df_brand['brand_eng'],y=df_brand['device_id'],data=df_brand)
plt.title('Distribution of Users across Phone Brands(Consider only 10 Most used Phone Brands).',fontsize=20)
plt.xlabel('Brand Name',fontsize=18,color='red')
plt.ylabel('No of records',fontsize=14)
plt.xticks(fontsize=14)
plt.show()

### 3. Distribution of Users across Gender.

In [ ]:
df_gender=data.groupby(by=['gender']).agg({'device_id':'nunique'}).reset_index().sort_values(by='device_id',ascending=False)

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x=df_gender['gender'],y=df_gender['device_id'],data=df_gender)
plt.title('Distribution of Users across Gender',fontsize=20)
plt.xlabel('Gender',fontsize=18,color='red')
plt.ylabel('No of records',fontsize=14)
plt.xticks(fontsize=14)
plt.show()

### 4. Distribution of Users across Age Segments.

In [ ]:
df_age_groups=data.groupby(by=['age_group']).agg({'device_id':'nunique'}).reset_index().sort_values(by='device_id',ascending=False)

In [ ]:
plt.figure(figsize=(15,10))
sns.barplot(x=df_age_groups['age_group'],y=df_age_groups['device_id'],data=df_age_groups)
plt.title('Distribution of Users across Age Segments',fontsize=20)
plt.xlabel('Age Group',fontsize=18,color='red')
plt.ylabel('No of records',fontsize=14)
plt.xticks(fontsize=14)
plt.show()

### 5. Distribution of Phone Brands  (Consider only 10 Most used Phone Brands) for each Age Segment, State, Gender.

In [ ]:
def distribution(data,colname):    
    val_list=data[colname].unique()
       
    df=pd.DataFrame()
    for val in val_list:
        
        df_val=data[data[colname]==val].groupby(by=['brand_eng']).agg({'device_id':'nunique'}).reset_index().sort_values(by='brand_eng')
        df_val.rename(columns={'device_id':val+'_count'},inplace=True)
        
        if df.empty:
            df=df_val.copy(deep=True)
        else:
            df=pd.merge(df,df_val,how='outer',left_on='brand_eng',right_on='brand_eng')
            
    df.fillna(0,inplace=True)
    df['total']=0
    for val in val_list:
        cname=val+'_count'
        df=df.astype({cname:'int'})          #df[val+'_count'].apply(pd.to_numeric())
        df['total']=df['total']+df[cname]
    
    
    df=df.sort_values(by='total',ascending=False)[:10].reset_index()
    df.drop(columns=['index','total'],inplace=True)
    df.set_index('brand_eng',inplace=True)
    
    plt.figure(figsize=(18,10))
    #Final_df.groupby(by='Season')['FTHG','FTAG'].sum().plot(kind='bar',figsize=(15,7))
    df.plot(kind='bar',figsize=(15,7))
    plt.xlabel('Brand Name',fontsize=16)
    plt.ylabel('No of Users',fontsize=16)
    plt.xticks(rotation=45,fontsize=14)
    plt.legend(val_list,fontsize=14)
    plt.title('Distribution of top 10 Phone Brands for '+ colname,fontsize=20)
    plt.show()
    

In [ ]:
distribution(data,'gender')

In [ ]:
distribution(data,'age_group')

In [ ]:
distribution(data,'state')

### 6. Distribution of Gender for each State, Age Segment and Phone Brand  (Consider only 10 Most used Phone Brands).

In [ ]:
def distributionGender(data,colname,lstBrand=[]):    
    datasource=data.copy(deep=True)
    if len(lstBrand)!=0:
        datasource=datasource[datasource['brand_eng'].isin(lstBrand)]
        
    val_list=datasource[colname].unique()
       
    df=pd.DataFrame()
    for val in val_list:
        
        df_val=datasource[datasource[colname]==val].groupby(by=['gender']).agg({'device_id':'nunique'}).reset_index().sort_values(by='gender')
        df_val.rename(columns={'device_id':val+'_count'},inplace=True)
        
        if df.empty:
            df=df_val.copy(deep=True)
        else:
            df=pd.merge(df,df_val,how='outer',left_on='gender',right_on='gender')
            
    df.fillna(0,inplace=True)
    df['total']=0
    for val in val_list:
        cname=val+'_count'
        df=df.astype({cname:'int'})          #df[val+'_count'].apply(pd.to_numeric())
        df['total']=df['total']+df[cname]
    
    
    df=df.sort_values(by='total',ascending=False).reset_index()
    df.drop(columns=['index','total'],inplace=True)
    df.set_index('gender',inplace=True)
    
    plt.figure(figsize=(18,10))
    #Final_df.groupby(by='Season')['FTHG','FTAG'].sum().plot(kind='bar',figsize=(15,7))
    df.plot(kind='bar',figsize=(15,7))
    plt.xlabel('Gender',fontsize=16)
    plt.ylabel('No of Users',fontsize=16)
    plt.xticks(rotation=0,fontsize=14)
    plt.legend(val_list,fontsize=14)
    if colname=='brand_eng':
        colname='Top 10 Brands'
    plt.title('Distribution of Gender for '+ colname,fontsize=20)
    plt.show()

In [ ]:
distributionGender(data,'state')

In [ ]:
distributionGender(data,'age_group')

In [ ]:
lstBrand=data.groupby(by=['brand_eng']).agg({'device_id':'nunique'}).reset_index().sort_values(by='device_id',ascending=False)[:10]['brand_eng'].values
lstBrand=lstBrand.tolist()

In [ ]:
distributionGender(data,'brand_eng',lstBrand)

### 7. Distribution of Age Segments for each State, Gender and Phone Brand  (Consider only 10 Most used Phone Brands).

In [ ]:
def distributionAgeSegment(data,colname,lstBrand=[]):    
    datasource=data.copy(deep=True)
    if len(lstBrand)!=0:
        datasource=datasource[datasource['brand_eng'].isin(lstBrand)]
        
    val_list=datasource[colname].unique()
       
    df=pd.DataFrame()
    for val in val_list:
        
        df_val=datasource[datasource[colname]==val].groupby(by=['age_group']).agg({'device_id':'nunique'}).reset_index().sort_values(by='age_group')
        df_val.rename(columns={'device_id':val+'_count'},inplace=True)
        
        if df.empty:
            df=df_val.copy(deep=True)
        else:
            df=pd.merge(df,df_val,how='outer',left_on='age_group',right_on='age_group')
            
    df.fillna(0,inplace=True)
    df['total']=0
    for val in val_list:
        cname=val+'_count'
        df=df.astype({cname:'int'})          #df[val+'_count'].apply(pd.to_numeric())
        df['total']=df['total']+df[cname]
    
    
    df=df.sort_values(by='total',ascending=False).reset_index()
    df.drop(columns=['index','total'],inplace=True)
    df.set_index('age_group',inplace=True)
    
    plt.figure(figsize=(18,10))
    #Final_df.groupby(by='Season')['FTHG','FTAG'].sum().plot(kind='bar',figsize=(15,7))
    df.plot(kind='bar',figsize=(18,11))
    plt.xlabel('Age Segment',fontsize=16)
    plt.ylabel('No of Users',fontsize=16)
    plt.xticks(rotation=45,fontsize=14)
    plt.legend(val_list,fontsize=14)
    if colname=='brand_eng':
        colname='Top 10 Brands'
    plt.title('Distribution of Age Segment for '+ colname,fontsize=20)
    plt.show()

In [ ]:
distributionAgeSegment(data,'state')

In [ ]:
distributionAgeSegment(data,'gender')

In [ ]:
distributionAgeSegment(data,'brand_eng',lstBrand)

### 8. Hourly distribution of Phone Calls.

In [ ]:
df_hour=data.sort_values(by='Hour')

In [ ]:
plt.figure(figsize=(18,11))
sns.histplot(x=df_hour['Hour'],data=df_hour)
plt.title('Hourly distribution of Phone Calls',fontsize=20)
plt.xlabel('Hour',fontsize=18,color='red')
plt.ylabel('No of records',fontsize=14)
plt.xticks(fontsize=14)
plt.show()

### 9. Plot the Users on the Map using any suitable package.

In [ ]:
def plotpackage(data,colname):
    inputval=input('Enter ' + colname + ' : ')
    df=data.groupby(by=['device_id',colname,'latitude','longitude']).agg({'age':'nunique'}).reset_index()
    df_filter=df[df[colname]==inputval]    
        
    map_osm = folium.Map(location=[21.7679, 78.8718], zoom_start=4.5, tiles="Stamen Toner")
    df_filter.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]], 
                                              radius=10,tooltip=row['device_id'],color="red")
                                             .add_to(map_osm), axis=1)    
    return map_osm

In [ ]:
plotpackage(data,'brand_eng')

In [ ]:
plotpackage(data,'age_group')